In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind

from backtrader.feeds import GenericCSVData

from statistics import mean
from statistics import median

In [2]:
class dix_csv(GenericCSVData):
    
    lines= ('dix', 'gex')
    
    params = (
        ('dtformat', '%m/%d/%Y'),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 6),
        ('openinterest', -1),
        ('dix', 9),
        ('gex', 10),        
    )

In [3]:
#Optimize DIX MA
class DIXmaopt(bt.Strategy):
    params = (
        ('period', 2),
        ('printlog', False),
        ('dixupper', 45),
        ('dixlower', 40),
        ('daystosell', 60)
    )
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #Add a DIXSMA Indicator
        self.dixsma = bt.indicators.MovingAverageSimple(
           self.data.dix, period=self.params.period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        #make a list of the pnl for each trade
        self.pnl_percent = trade.pnl / self.broker.getvalue()
        self.pnl_list=[]
        self.pnl_list.append(self.pnl_percent)

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # buy when dix greater than dixupper threshold
            if self.dixsma >= (self.params.dixupper/100):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        else:
            
            # sell after variable days
            self.duration = len(self) - self.bar_executed + 1
            if self.duration >= self.params.daystosell:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        #roi
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        #take the mean if a pnl exists
        try:
            self.pnl_mean = (mean(self.pnl_list) * 100)
        except:
            self.pnl_list = [0]
            self.pnl_mean = (mean(self.pnl_list) * 100)
#         #take the median if a pnl exists    
#         try:
#             self.pnl_median = (median(self.pnl_list) * 100)
#         except:
#             self.pnl_list = [0]
#             self.pnl_median = (median(self.pnl_list) * 100)
            
        print('(MA Period%2d) (DAYS TO SELL %d) (DIXUpper %2d) (MEAN TRADE PNL: %.3f%%) ROI %.2f%%' %
                 (self.params.period, self.params.daystosell, self.params.dixupper, self.pnl_mean, self.roi))
        
        
#runDIXma Strategy Opt
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()    
    
    # Add a strategy
    strats = cerebro.optstrategy(
        DIXmaopt,
        period=range(1,26),
        daystosell=range(30,91)
    ) 
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\DIX MA Testing Backtrader\DIX OHLC.csv')

    # Create a Data Feed
    data = dix_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run(maxcpus=1)
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    # Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


(MA Period 1) (DAYS TO SELL 30) (DIXUpper 45) (MEAN TRADE PNL: 0.192%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 31) (DIXUpper 45) (MEAN TRADE PNL: 0.248%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 32) (DIXUpper 45) (MEAN TRADE PNL: 0.403%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 33) (DIXUpper 45) (MEAN TRADE PNL: 0.415%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 34) (DIXUpper 45) (MEAN TRADE PNL: 0.307%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 35) (DIXUpper 45) (MEAN TRADE PNL: 0.177%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 36) (DIXUpper 45) (MEAN TRADE PNL: 0.264%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 37) (DIXUpper 45) (MEAN TRADE PNL: 0.240%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 38) (DIXUpper 45) (MEAN TRADE PNL: 0.174%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 39) (DIXUpper 45) (MEAN TRADE PNL: 0.200%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 40) (DIXUpper 45) (MEAN TRADE PNL: 0.147%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 41) (DIXUpper 45) (MEAN TRADE PNL: 0.285%) ROI 0.02%
(MA Period 1) (DAYS TO SELL 

(MA Period 2) (DAYS TO SELL 70) (DIXUpper 45) (MEAN TRADE PNL: 0.025%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 71) (DIXUpper 45) (MEAN TRADE PNL: -0.051%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 72) (DIXUpper 45) (MEAN TRADE PNL: 0.084%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 73) (DIXUpper 45) (MEAN TRADE PNL: 0.089%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 74) (DIXUpper 45) (MEAN TRADE PNL: 0.055%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 75) (DIXUpper 45) (MEAN TRADE PNL: 0.093%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 76) (DIXUpper 45) (MEAN TRADE PNL: 0.048%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 77) (DIXUpper 45) (MEAN TRADE PNL: 0.091%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 78) (DIXUpper 45) (MEAN TRADE PNL: 0.067%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 79) (DIXUpper 45) (MEAN TRADE PNL: 0.001%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 80) (DIXUpper 45) (MEAN TRADE PNL: 0.027%) ROI 0.02%
(MA Period 2) (DAYS TO SELL 81) (DIXUpper 45) (MEAN TRADE PNL: -0.027%) ROI 0.02%
(MA Period 2) (DAYS TO SEL

(MA Period 4) (DAYS TO SELL 50) (DIXUpper 45) (MEAN TRADE PNL: 0.648%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 51) (DIXUpper 45) (MEAN TRADE PNL: 0.559%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 52) (DIXUpper 45) (MEAN TRADE PNL: 0.508%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 53) (DIXUpper 45) (MEAN TRADE PNL: 0.432%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 54) (DIXUpper 45) (MEAN TRADE PNL: 0.568%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 55) (DIXUpper 45) (MEAN TRADE PNL: 0.574%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 56) (DIXUpper 45) (MEAN TRADE PNL: 0.539%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 57) (DIXUpper 45) (MEAN TRADE PNL: 0.578%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 58) (DIXUpper 45) (MEAN TRADE PNL: 0.532%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 59) (DIXUpper 45) (MEAN TRADE PNL: 0.576%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 60) (DIXUpper 45) (MEAN TRADE PNL: 0.551%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 61) (DIXUpper 45) (MEAN TRADE PNL: 0.484%) ROI 0.01%
(MA Period 4) (DAYS TO SELL 

(MA Period 5) (DAYS TO SELL 90) (DIXUpper 45) (MEAN TRADE PNL: -0.343%) ROI 0.01%
(MA Period 6) (DAYS TO SELL 30) (DIXUpper 45) (MEAN TRADE PNL: 0.186%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 31) (DIXUpper 45) (MEAN TRADE PNL: 0.189%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 32) (DIXUpper 45) (MEAN TRADE PNL: 0.410%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 33) (DIXUpper 45) (MEAN TRADE PNL: 0.444%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 34) (DIXUpper 45) (MEAN TRADE PNL: 0.449%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 35) (DIXUpper 45) (MEAN TRADE PNL: 0.465%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 36) (DIXUpper 45) (MEAN TRADE PNL: 0.443%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 37) (DIXUpper 45) (MEAN TRADE PNL: 0.499%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 38) (DIXUpper 45) (MEAN TRADE PNL: 0.510%) ROI 0.01%
(MA Period 6) (DAYS TO SELL 39) (DIXUpper 45) (MEAN TRADE PNL: 0.541%) ROI 0.02%
(MA Period 6) (DAYS TO SELL 40) (DIXUpper 45) (MEAN TRADE PNL: 0.520%) ROI 0.01%
(MA Period 6) (DAYS TO SELL

(MA Period 7) (DAYS TO SELL 69) (DIXUpper 45) (MEAN TRADE PNL: -0.552%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 70) (DIXUpper 45) (MEAN TRADE PNL: -0.460%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 71) (DIXUpper 45) (MEAN TRADE PNL: -0.454%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 72) (DIXUpper 45) (MEAN TRADE PNL: -0.432%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 73) (DIXUpper 45) (MEAN TRADE PNL: -0.441%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 74) (DIXUpper 45) (MEAN TRADE PNL: -0.437%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 75) (DIXUpper 45) (MEAN TRADE PNL: -0.394%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 76) (DIXUpper 45) (MEAN TRADE PNL: -0.391%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 77) (DIXUpper 45) (MEAN TRADE PNL: -0.451%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 78) (DIXUpper 45) (MEAN TRADE PNL: -0.448%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 79) (DIXUpper 45) (MEAN TRADE PNL: -0.426%) ROI 0.01%
(MA Period 7) (DAYS TO SELL 80) (DIXUpper 45) (MEAN TRADE PNL: -0.424%) ROI 0.01%
(MA Period 7) (D

...

(MA Period17) (DAYS TO SELL 70) (DIXUpper 45) (MEAN TRADE PNL: 0.322%) ROI 0.02%
(MA Period17) (DAYS TO SELL 71) (DIXUpper 45) (MEAN TRADE PNL: 0.318%) ROI 0.02%
(MA Period17) (DAYS TO SELL 72) (DIXUpper 45) (MEAN TRADE PNL: 0.333%) ROI 0.02%
(MA Period17) (DAYS TO SELL 73) (DIXUpper 45) (MEAN TRADE PNL: 0.304%) ROI 0.02%
(MA Period17) (DAYS TO SELL 74) (DIXUpper 45) (MEAN TRADE PNL: 0.311%) ROI 0.02%
(MA Period17) (DAYS TO SELL 75) (DIXUpper 45) (MEAN TRADE PNL: 0.291%) ROI 0.02%
(MA Period17) (DAYS TO SELL 76) (DIXUpper 45) (MEAN TRADE PNL: 0.295%) ROI 0.02%
(MA Period17) (DAYS TO SELL 77) (DIXUpper 45) (MEAN TRADE PNL: 0.262%) ROI 0.01%
(MA Period17) (DAYS TO SELL 78) (DIXUpper 45) (MEAN TRADE PNL: 0.243%) ROI 0.01%
(MA Period17) (DAYS TO SELL 79) (DIXUpper 45) (MEAN TRADE PNL: 0.246%) ROI 0.01%
(MA Period17) (DAYS TO SELL 80) (DIXUpper 45) (MEAN TRADE PNL: 0.223%) ROI 0.01%
(MA Period17) (DAYS TO SELL 81) (DIXUpper 45) (MEAN TRADE PNL: 0.204%) ROI 0.01%
(MA Period17) (DAYS TO SELL 

(MA Period19) (DAYS TO SELL 50) (DIXUpper 45) (MEAN TRADE PNL: 0.306%) ROI 0.01%
(MA Period19) (DAYS TO SELL 51) (DIXUpper 45) (MEAN TRADE PNL: 0.239%) ROI 0.01%
(MA Period19) (DAYS TO SELL 52) (DIXUpper 45) (MEAN TRADE PNL: 0.265%) ROI 0.01%
(MA Period19) (DAYS TO SELL 53) (DIXUpper 45) (MEAN TRADE PNL: 0.211%) ROI 0.01%
(MA Period19) (DAYS TO SELL 54) (DIXUpper 45) (MEAN TRADE PNL: 0.242%) ROI 0.01%
(MA Period19) (DAYS TO SELL 55) (DIXUpper 45) (MEAN TRADE PNL: 0.297%) ROI 0.01%
(MA Period19) (DAYS TO SELL 56) (DIXUpper 45) (MEAN TRADE PNL: 0.334%) ROI 0.01%
(MA Period19) (DAYS TO SELL 57) (DIXUpper 45) (MEAN TRADE PNL: 0.269%) ROI 0.01%
(MA Period19) (DAYS TO SELL 58) (DIXUpper 45) (MEAN TRADE PNL: 0.280%) ROI 0.01%
(MA Period19) (DAYS TO SELL 59) (DIXUpper 45) (MEAN TRADE PNL: 0.288%) ROI 0.01%
(MA Period19) (DAYS TO SELL 60) (DIXUpper 45) (MEAN TRADE PNL: 0.296%) ROI 0.01%
(MA Period19) (DAYS TO SELL 61) (DIXUpper 45) (MEAN TRADE PNL: 0.300%) ROI 0.01%
(MA Period19) (DAYS TO SELL 

(MA Period21) (DAYS TO SELL 30) (DIXUpper 45) (MEAN TRADE PNL: 0.224%) ROI 0.01%
(MA Period21) (DAYS TO SELL 31) (DIXUpper 45) (MEAN TRADE PNL: 0.238%) ROI 0.01%
(MA Period21) (DAYS TO SELL 32) (DIXUpper 45) (MEAN TRADE PNL: 0.264%) ROI 0.01%
(MA Period21) (DAYS TO SELL 33) (DIXUpper 45) (MEAN TRADE PNL: 0.297%) ROI 0.01%
(MA Period21) (DAYS TO SELL 34) (DIXUpper 45) (MEAN TRADE PNL: 0.309%) ROI 0.01%
(MA Period21) (DAYS TO SELL 35) (DIXUpper 45) (MEAN TRADE PNL: 0.361%) ROI 0.01%
(MA Period21) (DAYS TO SELL 36) (DIXUpper 45) (MEAN TRADE PNL: 0.396%) ROI 0.01%
(MA Period21) (DAYS TO SELL 37) (DIXUpper 45) (MEAN TRADE PNL: 0.409%) ROI 0.01%
(MA Period21) (DAYS TO SELL 38) (DIXUpper 45) (MEAN TRADE PNL: 0.409%) ROI 0.01%
(MA Period21) (DAYS TO SELL 39) (DIXUpper 45) (MEAN TRADE PNL: 0.321%) ROI 0.01%
(MA Period21) (DAYS TO SELL 40) (DIXUpper 45) (MEAN TRADE PNL: 0.269%) ROI 0.01%
(MA Period21) (DAYS TO SELL 41) (DIXUpper 45) (MEAN TRADE PNL: 0.193%) ROI 0.01%
(MA Period21) (DAYS TO SELL 

(MA Period22) (DAYS TO SELL 71) (DIXUpper 45) (MEAN TRADE PNL: 0.263%) ROI 0.01%
(MA Period22) (DAYS TO SELL 72) (DIXUpper 45) (MEAN TRADE PNL: 0.267%) ROI 0.01%
(MA Period22) (DAYS TO SELL 73) (DIXUpper 45) (MEAN TRADE PNL: 0.234%) ROI 0.01%
(MA Period22) (DAYS TO SELL 74) (DIXUpper 45) (MEAN TRADE PNL: 0.214%) ROI 0.01%
(MA Period22) (DAYS TO SELL 75) (DIXUpper 45) (MEAN TRADE PNL: 0.218%) ROI 0.01%
(MA Period22) (DAYS TO SELL 76) (DIXUpper 45) (MEAN TRADE PNL: 0.195%) ROI 0.01%
(MA Period22) (DAYS TO SELL 77) (DIXUpper 45) (MEAN TRADE PNL: 0.175%) ROI 0.01%
(MA Period22) (DAYS TO SELL 78) (DIXUpper 45) (MEAN TRADE PNL: 0.176%) ROI 0.01%
(MA Period22) (DAYS TO SELL 79) (DIXUpper 45) (MEAN TRADE PNL: 0.211%) ROI 0.01%
(MA Period22) (DAYS TO SELL 80) (DIXUpper 45) (MEAN TRADE PNL: 0.214%) ROI 0.01%
(MA Period22) (DAYS TO SELL 81) (DIXUpper 45) (MEAN TRADE PNL: 0.197%) ROI 0.01%
(MA Period22) (DAYS TO SELL 82) (DIXUpper 45) (MEAN TRADE PNL: 0.209%) ROI 0.01%
(MA Period22) (DAYS TO SELL 

(MA Period24) (DAYS TO SELL 51) (DIXUpper 45) (MEAN TRADE PNL: 0.206%) ROI 0.01%
(MA Period24) (DAYS TO SELL 52) (DIXUpper 45) (MEAN TRADE PNL: 0.261%) ROI 0.01%
(MA Period24) (DAYS TO SELL 53) (DIXUpper 45) (MEAN TRADE PNL: 0.298%) ROI 0.01%
(MA Period24) (DAYS TO SELL 54) (DIXUpper 45) (MEAN TRADE PNL: 0.115%) ROI 0.01%
(MA Period24) (DAYS TO SELL 55) (DIXUpper 45) (MEAN TRADE PNL: 0.107%) ROI 0.01%
(MA Period24) (DAYS TO SELL 56) (DIXUpper 45) (MEAN TRADE PNL: 0.105%) ROI 0.01%
(MA Period24) (DAYS TO SELL 57) (DIXUpper 45) (MEAN TRADE PNL: 0.110%) ROI 0.01%
(MA Period24) (DAYS TO SELL 58) (DIXUpper 45) (MEAN TRADE PNL: 0.114%) ROI 0.01%
(MA Period24) (DAYS TO SELL 59) (DIXUpper 45) (MEAN TRADE PNL: 0.094%) ROI 0.01%
(MA Period24) (DAYS TO SELL 60) (DIXUpper 45) (MEAN TRADE PNL: 0.096%) ROI 0.01%
(MA Period24) (DAYS TO SELL 61) (DIXUpper 45) (MEAN TRADE PNL: 0.096%) ROI 0.01%
(MA Period24) (DAYS TO SELL 62) (DIXUpper 45) (MEAN TRADE PNL: 0.106%) ROI 0.01%
(MA Period24) (DAYS TO SELL 